# 03: Feature selection, transformations

### Import

In [1]:
import numpy as np
import pandas as pd

from scipy.stats import norm, ttest_ind
from scipy.optimize import minimize_scalar

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import f_regression, mutual_info_regression, RFECV
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_log_error, make_scorer, mean_squared_error
from sklearn import preprocessing

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### Data load

In [2]:
# df = pd.read_csv('data.csv')

### Observe target variable

In [3]:
# Distribution of the target varaible
# sns.distplot(df.SalePrice, fit=norm)

In [4]:
# # Observe skewness and kurtosis as measures of asymetry and long tails
# print(df.SalePrice.skew())
# print(df.SalePrice.kurtosis())

# # Check log transform
# print(np.log1p(df.SalePrice).skew())
# print(np.log1p(df.SalePrice).kurtosis())

In [5]:
# Try to transform to obtain normal distribution
# sns.distplot(np.log1p(df.SalePrice), fit=norm)

In [6]:
# Log transform of SalePrices
# df.SalePrice = np.log1p(df.SalePrice)

### Simple clean & transform

In [7]:
# Remove Miscellaneous feature value since we don't want to create several value variables for each feature (try it at home)
# df.drop(['MiscVal'], axis=1, inplace = True, errors = 'ignore')

In [8]:
# show features with largest number of missing values
# display(df.isnull().sum().nlargest(5))
# # find features with more than half missings
# half_miss_columns = df.columns[df.isnull().sum() > len(df)/2]
# # drop
# df.drop(half_miss_columns, axis = 1, inplace = True)
# # see result
# display(df.isnull().sum().nlargest(5))

In [9]:
# # Fireplaces are connected with fireplace quality - i.e data are cleaned
# df[df.Fireplaces != 0].FireplaceQu.isnull().sum()

In [10]:
# Fill all NaN with 0
# df.fillna(0, inplace = True)

In [11]:
# This point is not necessary for later get_dummies
# See unique values for object type
# display(df.select_dtypes(include=['object']).nunique().nlargest(5))
# # Convert all object values to categorial format
# df[df.select_dtypes(include=['object']).columns] = df.select_dtypes(include=['object']).apply(pd.Series.astype, dtype='category')

In [12]:
# Convert all Area and Square feet to log values
# for column in df.filter(regex='Area|SF', axis=1).columns:
#     df['Has' + column] = (df[column] > 0).replace({True: 1, False: 0}).astype('uint8')
# #     df.loc[df[column] > 0, column] = np.log(df.loc[df[column] > 0, column])
#     df['Sqrt' + column] = np.sqrt(df[column])
#     df['Log' + column] = np.log1p(df[column])

In [13]:
# df[df.filter(regex='Area', axis=1).columns].head()

In [14]:
# sns.distplot(df[df.LotArea>0].LogLotArea, fit = norm)

In [15]:
# Convert categorical variables to indicators and create new data
# # This is usefull for regression (resonable model) - in the case of classification this is usually not necessary 
# df = pd.get_dummies(df)
# # see how many of each type we have
# df.dtypes.value_counts()

In [16]:
# Remove constant features
# display(df.columns[df.min() == df.max()])
# df = df[df.columns[df.min() != df.max()]]

In [17]:
# collect type column names
# continuous_columns = df.select_dtypes(include=['float64']).columns
# discrete_columns = df.select_dtypes(include=['int64']).columns
# indicator_columns = df.select_dtypes(include=['uint8']).columns

### Split train & test data to demonstrate results

In [18]:
# dt, dtest = train_test_split(df, test_size=0.25, random_state=17)
# dt = dt.copy()
# dtest = dtest.copy()

In [19]:
# print('Train: ', len(dt), '; Test: ', len(dtest))

## Normalize features

In [20]:
# Check max and min values for indicators  - should be stored in uint8
# print(dt.select_dtypes(include=['uint8']).min().min())
# print(dt.select_dtypes(include=['uint8']).max().max())

In [21]:
# # Standardize scaling
# scaler = preprocessing.StandardScaler()

# # Prepare column names - ['float64', 'int64', 'uint8']
# columns = dt.select_dtypes(include=['float64', 'int64']).columns
# columns = columns.drop('SalePrice', errors = 'ignore')
# print(list(columns))

# # Prepare values for transform
# scaler.fit(dt[columns])

# # Transform
# dt[columns] = scaler.transform(dt[columns])
# dtest[columns] = scaler.transform(dtest[columns])

In [22]:
# dt.dtypes.value_counts()

In [23]:
# sns.boxplot(data=dt[['OverallQual']])

## Feature selection

### Find irelevant features

In [24]:
# # Drop Id
# dt.drop(['Id'], axis = 1, errors = 'ignore', inplace = True)
# dtest.drop(['Id'], axis = 1, errors = 'ignore', inplace = True)

In [25]:
# # Find features with low variance
# columns_to_remove = dt.columns[dt.var() < 0.02]
# dt[columns_to_remove].var().nlargest(5)
# print(len(columns_to_remove))

In [26]:
# # Perform t-test with indicator variables - calculate p-values
# ttest_pvals = df\
#     .drop(columns_to_remove, axis = 1, errors = 'ignore')\
#     .select_dtypes(include = ['uint8']).columns\
#     .to_series()\
#     .apply(lambda x: ttest_ind(df.SalePrice[df[x] == 0], df.SalePrice[df[x] == 1], equal_var = False).pvalue)

# # show largest and smallest
# display(ttest_pvals.nlargest(5))
# display(ttest_pvals.nsmallest(4))

In [27]:
# fig, axs = plt.subplots(ncols=2, figsize=(15,7))
# sns.boxplot(x='Exterior1st_Plywood', y='SalePrice', data=dt,ax=axs[0])
# sns.boxplot(x='ExterQual_TA', y='SalePrice', data=dt,ax=axs[1])

In [28]:
# # Remove all largest than 50%
# columns_to_remove = list(set(columns_to_remove).union(set(ttest_pvals[ttest_pvals > 0.5].index)))
# print(len(columns_to_remove))

### Correlation approach

In [29]:
# # Correlation matrix Pearson Spearman
# corrP = dt.drop(columns_to_remove, axis = 1, errors = 'ignore').corr(method='pearson')
# corrS = dt.drop(columns_to_remove, axis = 1, errors = 'ignore').corr(method='spearman')

In [30]:
# # Pearson top 10
# corrP_cols = corrP.SalePrice.abs().nlargest(10).index
# display(corrP.SalePrice.loc[corrP_cols])
# # Spearman top 10
# corrS_cols = corrS.SalePrice.abs().nlargest(10).index
# display(corrS.SalePrice.loc[corrS_cols])

In [31]:
# # Plot those correlations
# fig, axs = plt.subplots(ncols=2, figsize=(15,7))
# sns.heatmap(corrP.abs().loc[corrP_cols,corrP_cols],ax=axs[0], cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=corrP_cols.values, xticklabels=corrP_cols.values)
# sns.heatmap(corrS.abs().loc[corrS_cols,corrS_cols],ax=axs[1], cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=corrS_cols.values, xticklabels=corrS_cols.values)

In [32]:
# # OverallQual is actually orderd categorical variable stored as number
# sns.boxplot(x='OverallQual', y='SalePrice', data=dt)

### Subset selection

In [33]:
# # Use scikit learn to select best features
# # prepare data
# X = dt.drop(columns_to_remove, axis = 1, errors = 'ignore').drop(['SalePrice'], axis = 1, errors = 'ignore')
# y = dt.SalePrice

# # calculate F values for correlation coefficient (calculated from rho using an increasing function)
# # calculate p-values of the correspondong F-test (inversely proportional to rho)
# Fscores, pvals = f_regression(X,y)
# print(max(pvals))

In [34]:
# display(pd.Series(pvals, index=X.columns).nsmallest(5))
# display(corrP.SalePrice.abs().nlargest(5))

In [35]:
# # We can also use Mutual Information (information gain) - but for linear regression - correlation is usually better
# mi = mutual_info_regression(X,y)
# display(pd.Series(mi, index=X.columns).nlargest(5))

In [36]:
# # Stupid selection of n performing highest correlations
# n = 100
# columns_to_leave = corrP.SalePrice.abs().nlargest(n).index

In [37]:
# # Select by linear regression
# X = dt.drop(columns_to_remove, axis = 1, errors = 'ignore').drop(['SalePrice'], axis = 1, errors = 'ignore')
# # X = dt.drop(['SalePrice'], axis = 1, errors = 'ignore')
# y = dt.SalePrice

# used_columns = X.columns

# def scorer(Y, yp):
#     return np.sqrt(mean_squared_error(Y, yp))

# clfM = LinearRegression()

# selector = RFECV(clfM, step=1, cv=5, scoring=make_scorer(scorer))
# selector = selector.fit(X, y)

In [38]:
# # transform result to dataframe
# result = pd.DataFrame({'Chosen': selector.support_, 'Ranking': selector.ranking_}, index=list(used_columns))
# # columns to leave
# columns_to_leave = result[result.Chosen == True].index
# # show results
# display(result[result.Chosen == False].head(5))
# display(result.Chosen.sum())

## Estimation - Linear Regression

In [39]:
# # Data prepare
# X = dt[columns_to_leave].drop(['SalePrice'], axis = 1, errors = 'ignore')
# y = dt.SalePrice
# Xtest = dtest[columns_to_leave].drop(['SalePrice'], axis = 1, errors = 'ignore')
# ytest = dtest.SalePrice

# # Linear Regression
# clf1 = LinearRegression()
# clf1.fit(X, y) 

# # Print RMSLE
# print('Root mean squared logarithmic error:', np.sqrt(mean_squared_error(clf1.predict(Xtest), ytest)))

# # Joint Plot
# sns.jointplot(ytest, clf1.predict(Xtest))

### Estimation  - Ridge Regression

In [40]:
# # Data prepare with all features
# X = dt.drop(['SalePrice'], axis = 1, errors = 'ignore')
# # X = X.drop(columns_to_remove, axis = 1, errors = 'ignore')
# y = dt.SalePrice

# Xtest = dtest.drop(['SalePrice'], axis = 1, errors = 'ignore')
# # Xtest = Xtest.drop(columns_to_remove, axis = 1, errors = 'ignore')
# ytest = dtest.SalePrice

# # Determine alpha using cross validation
# def scorer(Y, yp):
#     return np.sqrt(mean_squared_error(Y, yp))

# def fun(alpha):
#     clf = Ridge(alpha=alpha)
#     return np.mean(cross_val_score(clf, X, y, cv=5, scoring=make_scorer(scorer)))

# # minimize
# opt_alpha = minimize_scalar(fun, options = {'maxiter': 30}, method = 'bounded', bounds=(0.1, 400))
# print(opt_alpha)

# # Ridge regression
# clf2 = Ridge(alpha=opt_alpha.x)
# clf2.fit(X, y) 

# # Print RMSLE
# print('Root mean squared logarithmic error:', np.sqrt(mean_squared_error(clf2.predict(Xtest), ytest)))

# # Joint Plot
sns.jointplot(ytest, clf2.predict(Xtest))

NameError: name 'ytest' is not defined

### Estimation - Kernel Regression

In [ ]:
# # Data prepare with all features
# X = dt.drop(['SalePrice'], axis = 1, errors = 'ignore')
# # X = X.drop(columns_to_remove, axis = 1, errors = 'ignore')
# y = dt.SalePrice

# Xtest = dtest.drop(['SalePrice'], axis = 1, errors = 'ignore')
# # Xtest = Xtest.drop(columns_to_remove, axis = 1, errors = 'ignore')
# ytest = dtest.SalePrice

# # Determine alpha using cross validation
# def scorer(Y, yp):
#     return np.sqrt(mean_squared_error(Y, yp))

# def fun(alpha):
#     clf = KernelRidge(alpha=alpha)
#     return np.mean(cross_val_score(clf, X, y, cv=5, scoring=make_scorer(scorer)))

# # minimize
# opt_alpha = minimize_scalar(fun, options = {'maxiter': 40}, method='bounded', bounds=(0.001, 100))
# print(opt_alpha)

# # Ridge regression
# clf3 = KernelRidge(alpha=opt_alpha.x)
# clf3.fit(X, y) 

# # Print RMSLE
# print('Root mean squared logarithmic error:', np.sqrt(mean_squared_error(clf3.predict(Xtest), ytest)))

# # Joint Plot
# sns.jointplot(ytest, clf3.predict(Xtest))

## Homework task 3:
* Load homework_data.csv.
* Analyze the distribution of 'price' and try to transform it into normal distibution.
* Perform proper feature transformations.
* Remove all irelevant features.
* Select a proper subset of features.
* Try to construct linear regression model to predict prices. Evaluate its root mean squared logarithmic error.

In [ ]:
# dh = pd.read_csv('homework_data.csv')